In [1]:
import sys
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import re
from fake_useragent import UserAgent
from functions import *

In [2]:


url = "https://tez.yok.gov.tr/UlusalTezMerkezi/"



In [3]:
üniversiteler = pd.read_csv("üniversiteler.csv")['üniversiteler'].tolist()
üniversiteler[185] = 'Melikşah Üniversitesi'

# 2025 yılı itibari ile eklenen yeni üniversite sadece Mudanya Üniversitesi ve hiç Tıpta Uzmanlık Tezi vermemiştir.

In [4]:
üniversiteler.index("Tekirdağ Namık Kemal Üniversitesi")

234

In [5]:
print(üniversiteler[150])

İzmir Tınaztepe Üniversitesi


In [ ]:
import random
import time
import logging
from datetime import datetime
import os
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from fake_useragent import UserAgent

# Logging ayarları
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("scraper.log"),
        logging.StreamHandler()
    ]
)

# Sabitleri tanımla
year1 = 2023
year2 = 2025  # yıl2 yıl1 den büyük olmalı
DATA_DIR = "data"

# Veri dizini oluştur (eğer yoksa)
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# İşlem izleme için kayıt dosyası
processed_unis_file = os.path.join(DATA_DIR, "processed_universities.txt")
failed_unis_file = os.path.join(DATA_DIR, "failed_universities.txt")

def get_processed_unis():
    """İşlenmiş üniversitelerin listesini döndürür"""
    if os.path.exists(processed_unis_file):
        with open(processed_unis_file, "r", encoding="utf-8") as f:
            return [line.strip() for line in f.readlines()]
    return []

def mark_as_processed(uni):
    """Üniversiteyi işlenmiş olarak işaretle"""
    with open(processed_unis_file, "a", encoding="utf-8") as f:
        f.write(f"{uni}\n")

def mark_as_failed(uni, reason):
    """Üniversiteyi başarısız olarak işaretle"""
    with open(failed_unis_file, "a", encoding="utf-8") as f:
        f.write(f"{uni}: {reason} - {datetime.now()}\n")

def random_sleep(min_time=2, max_time=5):
    """Rastgele bekleme süresi"""
    sleep_time = random.uniform(min_time, max_time)
    logging.debug(f"Sleeping for {sleep_time:.2f} seconds")
    time.sleep(sleep_time)

def process_university(üni):
    """Bir üniversite için tüm işlemleri gerçekleştirir"""
    logging.info(f"Processing university: {üni}")
    random_sleep(2, 5)
    
    driver = None
    try:
        # Yeni bir driver başlat
        driver = get_driver(url)
        
        # Üniversite ve yıl seçimi yap
        logging.info(f"Selecting university {üni} and years {year1}-{year2}")
        click_process(driver, üni, yıl1=year1, yıl2=year2)
        
        sonuç = sonuç_sayısı(driver)
        logging.info(f"{üni} için sonuç sayısı: {sonuç}")
        
        if not sonuç or sonuç == 0:
            logging.warning(f"No results for {üni}")
            return
        
        if sonuç < 2000:
            # Sonuç sayısı 2000'den az, doğrudan işle
            process_results(driver, üni, year1, year2)
        elif sonuç < 4000:
            # Driver'ı kapat ve yıl aralıklarına bölerek işle
            if driver:
                driver.close()
                driver = None
            process_larger_results(üni, sonuç, 12, 2000)
        else:
            # Driver'ı kapat ve daha küçük yıl aralıklarına bölerek işle
            if driver:
                driver.close()
                driver = None
            process_larger_results(üni, sonuç, 6, 1000)
        
        mark_as_processed(üni)
        logging.info(f"{üni} için işlem tamamlandı")
        logging.info("-----------------------------------------------------------")
    
    except Exception as e:
        logging.error(f"Error processing {üni}: {str(e)}")
        mark_as_failed(üni, str(e))
    finally:
        # Eğer hala açık bir driver varsa kapatıyoruz
        if driver:
            try:
                driver.close()
                driver = None
            except Exception as e:
                logging.warning(f"Error closing driver: {str(e)}")

def process_results(driver, üni, yıl1, yıl2):
    """Sonuçları işleyip CSV olarak kaydeder"""
    logging.info(f"Processing results for {üni} ({yıl1}-{yıl2})")
    
    try:
        df = get_table(driver)
        en, tr = [], []
        
        total_rows = len(df)
        for idx, rows in df.iterrows():
            logging.debug(f"Processing row {idx+1}/{total_rows}")
            # Her abstract arasında rastgele bekleme ekle
            random_sleep(0.5, 1.5)  
            try:
                e, t = get_abstract(driver, rows["tez_detay"], rows["tez_no"])
                en.append(e)
                tr.append(t)
            except Exception as e:
                logging.warning(f"Error getting abstract for thesis {rows['tez_no']}: {str(e)}")
                en.append("")
                tr.append("")
        
        df['en'] = en
        df['tr'] = tr
        df.drop(columns=['tez_detay'], inplace=True)
        
        output_file = os.path.join(DATA_DIR, f"{üni}_{yıl1}_{yıl2}.csv")
        df.to_csv(output_file, index=False)
        logging.info(f"Saved data to {output_file}")
        
    except Exception as e:
        logging.error(f"Error processing results: {str(e)}")
        raise

def process_larger_results(üni, sonuç, year_decrement, threshold):
    """Büyük sonuç kümelerini yıl aralıklarına bölerek işler"""
    döngü = sonuç // threshold + 2
    
    for i in range(1, döngü):
        logging.info(f"Toplam sonuç sayısı: {threshold}'den fazla olduğu için yıl aralığı seçiliyor")
        random_sleep()
        
        if i == döngü - 1:
            yıl1 = 1900
        else:
            yıl1 = year1 - year_decrement * i
        yıl2 = year2
        
        logging.info(f"Yıl aralığı: {yıl1}-{yıl2}")
        
        driver = None
        try:
            # Her yıl aralığı için yeni bir driver başlat
            driver = get_driver(url)
            click_process(driver, üni, yıl1=yıl1, yıl2=yıl2)
            sonuç = sonuç_sayısı(driver)
            
            if sonuç == 0:
                logging.info(f"No results for {üni} in year range {yıl1}-{yıl2}")
                continue
                
            logging.info(f"{üni} için sonuç sayısı: {sonuç}")
            process_results(driver, üni, yıl1, yıl2)
            
        except Exception as e:
            # Hata durumunda loglayıp devam et
            logging.error(f"Error in year range {yıl1}-{yıl2} for {üni}: {str(e)}")
        finally:
            # Her döngüde driver'ı temizliyoruz
            if driver:
                try:
                    driver.close()
                    driver = None
                except Exception as e:
                    logging.warning(f"Error closing driver: {str(e)}")
        
        # Ardışık istekler arasında daha uzun bekleme
        random_sleep(3, 7)

def main():
    processed_unis = get_processed_unis()
    remaining_unis = [üni for üni in üniversiteler[157:] if üni not in processed_unis]
    
    logging.info(f"Starting scraper. Total universities to process: {len(remaining_unis)}")
    
    for üni in remaining_unis:
        process_university(üni)
        # Üniversiteler arası uzun bekleme
        random_sleep(5, 10)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        logging.info("Script interrupted by user")
    except Exception as e:
        logging.critical(f"Unexpected error: {str(e)}")

2025-03-15 14:05:14,613 - INFO - Starting scraper. Total universities to process: 104
2025-03-15 14:05:14,614 - INFO - Processing university: Kara Harp Okulu Komutanlığı
2025-03-15 14:05:19,419 - INFO - patching driver executable /home/serdar/.local/share/undetected_chromedriver/undetected_chromedriver
2025-03-15 14:05:24,018 - INFO - Selecting university Kara Harp Okulu Komutanlığı and years 2023-2025
2025-03-15 14:05:38,945 - INFO - Kara Harp Okulu Komutanlığı için sonuç sayısı: 0
2025-03-15 14:05:38,945 - WARNING - No results for Kara Harp Okulu Komutanlığı
2025-03-15 14:05:44,059 - INFO - Processing university: Karadeniz Teknik Üniversitesi
2025-03-15 14:05:49,093 - INFO - patching driver executable /home/serdar/.local/share/undetected_chromedriver/undetected_chromedriver
2025-03-15 14:05:53,656 - INFO - Selecting university Karadeniz Teknik Üniversitesi and years 2023-2025
2025-03-15 14:06:08,600 - INFO - Karadeniz Teknik Üniversitesi için sonuç sayısı: 198
2025-03-15 14:06:08,601

# Sonuç Sayısı fazla olanları çekme

In [ ]:
# sağlık bilimleri üniversitesi 2017-2023 arası 10000 sonuç var. yıl yıl çekilmeli.

In [ ]:
import sys
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import re
from fake_useragent import UserAgent
from functions import *

In [ ]:


url = "https://tez.yok.gov.tr/UlusalTezMerkezi/"



In [ ]:
driver = get_driver(url)


In [ ]:
df = get_table(driver)
en, tr = [], []
for idx,rows in df.iterrows():
    e, t = get_abstract(driver,rows["tez_detay"],rows["tez_no"])
    en.append(e)
    tr.append(t)
df['en'] = en
df['tr'] = tr
df.drop(columns=['tez_detay'], inplace=True)


# CSV adını değiştirmeyi unutma
df.to_csv("Diş Hekimliği Uzmanlık_2016_2020.csv", index=False)
driver.close()